<a href="https://colab.research.google.com/github/ShoAnn/indo-legal/blob/main/test_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Model (Trained)

In [1]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

base_model = SentenceTransformer("indobenchmark/indobert-base-p1")
model = SentenceTransformer("ShoAnn/retriever_checkpoints")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/709k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

#Load and Prepare Test Dataset

In [2]:
!pip install -q datasets
from datasets import load_dataset
import pandas as pd
import torch

qpn_dataset = load_dataset('ShoAnn/indo_legalqa_triplets')

lb_all = pd.read_csv(
    "/content/drive/MyDrive/SKRIPSI/lb_v1_5474.csv",
    index_col=0,
    dtype={
        "pasal":"Int64",
        "angka":"Int64",
        "nomor":"Int64",
        "tahun":"Int64",
    })
queries = list(set(qpn_dataset["test"]["q"]))
corpus = list(lb_all['fulltext'])

corpus_embeddings_base = base_model.encode(corpus, convert_to_tensor=True)
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

README.md:   0%|          | 0.00/629 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/319k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4876 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/610 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/610 [00:00<?, ? examples/s]

#Test loop

In [3]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
from tqdm import tqdm

def retrieve_docs(queries, top_k, model, corpus_embeddings):
    retrieved_docs = []
    for query in queries:
        query_embedding = model.encode(query, convert_to_tensor=True)

        similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
        scores, indices = torch.topk(similarity_scores, k=top_k)

        docs = []
        for score, idx in zip(scores, indices):
            docs.append((corpus[idx], score.detach().cpu().item()))
        retrieved_docs.append({"query": query, "docs": docs})
    return retrieved_docs

In [4]:
k5_base = pd.DataFrame(retrieve_docs(queries, 5, base_model, corpus_embeddings_base))
k10_base = pd.DataFrame(retrieve_docs(queries, 10, base_model, corpus_embeddings_base))
k15_base = pd.DataFrame(retrieve_docs(queries, 15, base_model, corpus_embeddings_base))
k20_base = pd.DataFrame(retrieve_docs(queries, 20, base_model, corpus_embeddings_base))

k5 = pd.DataFrame(retrieve_docs(queries, 5, model, corpus_embeddings))
k10 = pd.DataFrame(retrieve_docs(queries, 10, model, corpus_embeddings))
k15 = pd.DataFrame(retrieve_docs(queries, 15, model, corpus_embeddings))
k20 = pd.DataFrame(retrieve_docs(queries, 20, model, corpus_embeddings))

#Compute Metrics

In [11]:
class TestSet:
    def __init__(self, ground_truth_df, test_df):
        self.ground_truth_df = ground_truth_df
        self.test_df = test_df
    def __recall(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        tp = [doc for doc in test_docs if doc in gt_list]
        recall = len(tp) / len(gt_list) # TP / (TP + FN) where FN = len(gt_list) - len(tp)
        return recall

    def get_recall_score(self, k):
        recall = 0
        for _, row in self.test_df.iterrows():
            recall += self.__recall(row)
        return recall / len(self.test_df)

    def __reciprocal_rank(self, row):
        query = row["query"]
        test_docs = [doc[0] for doc in row["docs"]]
        gt = self.ground_truth_df[self.ground_truth_df["query"] == query]
        gt_list = gt["docs"].iloc[0]

        for rank, doc_id in enumerate(test_docs, 1):
            if doc_id in gt_list:
                return 1.0 / rank
        return 0.0

    def get_mrr_score(self, k):
        rr_sum = 0
        for _, row in self.test_df.iterrows():
            rr_sum += self.__reciprocal_rank(row)
        return rr_sum / len(self.test_df) if len(self.test_df) > 0 else 0.0


In [6]:
ground_truth = pd.DataFrame(set(qpn_dataset["test"]["q"]), columns=["query"])
def group_pos(query):
    pos_list = []
    for i in range(len(qpn_dataset["test"]["q"])):
        if qpn_dataset["test"]["q"][i] == query:
            pos_list.append(qpn_dataset["test"]["pos"][i])
    return pos_list
ground_truth["docs"] = ground_truth["query"].apply(group_pos)
ground_truth

,query,docs
0,Apakah saksi ahli dalam perkara korupsi dapat ...,[7000 – KEWAJIBAN MEMBUAT LAPORAN Auditor inve...
1,Bagaimana pandangan hukum terhadap saksi korba...,[['(1) dalam hal saksi atau ahli tanpa alasan...
2,Teman saya meminjamkan BPKB mobilnya kepada te...,[Tiap jumlah maksimum hukuman denda yang dianc...
3,Pemerintah desa saya sedang mengambil kebijaka...,[(1) Klasifikasi belanja Desa terdiri atas bid...
4,Pacar saya sudah putus dengan mantannya 6 bula...,[Tiap jumlah maksimum hukuman denda yang dianc...
...,...,...
443,Apakah hukuman boleh dibatalkan saat sidang pe...,[pasal 51 tindak pidana kekerasan fisik seba...
444,Apa perbedaan pidana kurungan dan penjara? Moh...,[Undang-Undang ini mulai berlaku setelah 3 (ti...
445,Bisakah orang dipidana karena berduaan dengan ...,[Pasal 624 Undang-Undang ini mulai berlaku set...
446,"Teman saya melanggar lalu lintas, dengan tidak...",[Yang dimaksud dengan tindak pidana Undang-Und...


## base model

In [7]:
test_k_5_base = TestSet(ground_truth, k5_base)
test_k_10_base = TestSet(ground_truth, k10_base)
test_k_15_base = TestSet(ground_truth, k15_base)
test_k_20_base = TestSet(ground_truth, k20_base)

recall_k_5_base = test_k_5_base.get_recall_score(5)
recall_k_10_base = test_k_10_base.get_recall_score(10)
recall_k_15_base = test_k_15_base.get_recall_score(15)
recall_k_20_base = test_k_15_base.get_recall_score(20)

print(f"Recall@5: {recall_k_5_base}")
print(f"Recall@10: {recall_k_10_base}")
print(f"Recall@15: {recall_k_15_base}")
print(f"Recall@20: {recall_k_20_base}")

Recall@5: 0.028050595238095243
Recall@10: 0.05632440476190475
Recall@15: 0.06376488095238095
Recall@20: 0.06376488095238095


In [13]:
mrr_k_5_base = test_k_5_base.get_mrr_score(5)
mrr_k_10_base = test_k_10_base.get_mrr_score(10)
mrr_k_15_base = test_k_15_base.get_mrr_score(15)
mrr_k_20_base = test_k_15_base.get_mrr_score(20)

print(f"mrr@5: {mrr_k_5_base}")
print(f"mrr@10: {mrr_k_10_base}")
print(f"mrr@15: {mrr_k_15_base}")
print(f"mrr@20: {mrr_k_20_base}")

mrr@5: 0.02566964285714286
mrr@10: 0.029530718537414964
mrr@15: 0.030217271567717998
mrr@20: 0.030217271567717998


## fine tuned model

In [8]:
test_k_5 = TestSet(ground_truth, k5)
test_k_10 = TestSet(ground_truth, k10)
test_k_15 = TestSet(ground_truth, k15)
test_k_20 = TestSet(ground_truth, k20)

recall_k_5 = test_k_5.get_recall_score(5)
recall_k_10 = test_k_10.get_recall_score(10)
recall_k_15 = test_k_15.get_recall_score(15)
recall_k_20 = test_k_15.get_recall_score(20)
print(f"Recall@5: {recall_k_5}")
print(f"Recall@10: {recall_k_10}")
print(f"Recall@15: {recall_k_15}")
print(f"Recall@20: {recall_k_20}")

Recall@5: 0.0
Recall@10: 0.005208333333333334
Recall@15: 0.005952380952380953
Recall@20: 0.005952380952380953


In [16]:
mrr_k_5 = test_k_5.get_mrr_score(5)
mrr_k_10 = test_k_10.get_mrr_score(10)
mrr_k_15 = test_k_15.get_mrr_score(15)
mrr_k_20 = test_k_15.get_mrr_score(20)

print(f"mrr@5: {mrr_k_5}")
print(f"mrr@10: {mrr_k_10}")
print(f"mrr@15: {mrr_k_15}")
print(f"mrr@20: {mrr_k_20}")

mrr@5: 0.0
mrr@10: 0.000496031746031746
mrr@15: 0.000729955808080808
mrr@20: 0.000729955808080808


## Recall

## MRR